* Основной чекпойнт для этой задачи можно скачать по ссылке: https://drive.google.com/drive/folders/1ywaDZ4xX_ahcf0jWMJX69TDMdMhMqtmY?usp=sharing
    * Предполагается, что после загрузки лежат в папке all_data/general_checkpoints
    * 

* Данные для обучения по ссылке (кладутся в основную папку all_data): https://drive.google.com/drive/folders/1oHRzegJm9tId0jliPQPq5G7bYwtwVEK7?usp=sharing



In [1]:
from scripts.data_utils.dataset_process import get_ind_sequence, dataset
from scripts.data_utils.label_helper import labels2ids, ids2labels
from transformers import get_linear_schedule_with_warmup
from scripts.task_trainers import GeneralTrainer
from scripts.span_classifier import NNERModel
from torch.utils.data import DataLoader
from torch import cuda, optim
import pandas as pd
import numpy as np
import random
import torch
import re


In [2]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [3]:
def set_seed(SEED=42):
    
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    
set_seed()

In [4]:
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


##### Загрузка данных для обучения и валидации

Везде требуется уточнить пути до файлов

train

In [5]:
# тренировочные тексты
ner_ds_path = 'all_data/train_texts.csv'

train_texts = pd.read_csv(ner_ds_path, sep=';')
to_del = [idx for idx, sent in enumerate(train_texts.Contents) if re.match(r'^\s+$', sent)]
train_texts = train_texts.drop(to_del).reset_index()
train_texts = train_texts.drop('index', axis=1)
# получение границ слов в последовательности
train_inds = get_ind_sequence(train_texts)

# тренировочные лейблы
ner_ds_path = 'all_data/train_spans.csv'
train_spans = pd.read_csv(ner_ds_path, sep=';')

In [6]:
train_texts_ds = dataset(train_texts, max_len=284)

val

In [7]:
ner_ds_path = 'all_data/dev_texts.csv'

val_texts = pd.read_csv(ner_ds_path, sep=';')
to_del = [idx for idx, sent in enumerate(val_texts.Contents) if re.match(r'^\s+$', sent)]
val_texts = val_texts.drop(to_del).reset_index()
val_texts = val_texts.drop('index', axis=1)
val_inds = get_ind_sequence(val_texts)

ner_ds_path = 'all_data/dev_spans.csv'
val_spans = pd.read_csv(ner_ds_path, sep=';')

In [8]:
val_texts_ds = dataset(val_texts, max_len=284)

test

In [9]:
ner_ds_path = 'all_data/test_texts.csv'

test_texts = pd.read_csv(ner_ds_path, sep=';')
to_del = [idx for idx, sent in enumerate(test_texts.Contents) if re.match(r'^\s+$', sent)]
test_texts = test_texts.drop(to_del).reset_index()
test_texts = test_texts.drop('index', axis=1)
test_inds = get_ind_sequence(test_texts)

ner_ds_path = 'all_data/test_spans.csv'
test_spans = pd.read_csv(ner_ds_path, sep=';')

In [10]:
test_texts_ds = dataset(test_texts, max_len=284)

##### Обучение

In [11]:
TRAIN_BATCH_SIZE = 1
VALID_BATCH_SIZE = 1

train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

val_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

test_params = {'batch_size': 1,
                'shuffle': False,
                'num_workers': 0
                }


training_loader = DataLoader(train_texts_ds, **train_params)
validation_loader = DataLoader(val_texts_ds, **val_params)
testing_loader = DataLoader(test_texts_ds, **test_params)

In [12]:
EPOCHS = 100
LEARNING_RATE = 1e-05

NNERModel - модель для основной задачи классификации вложенных сущностей


Важные параметры:

* extractor_type - тип используемого преобразования для получения представлений отрезков

    * 'weightedpooling' - простой взвещенный пулинг BERT-представлений слов
    * 'lstmattention' - модуль многоголового внимания со слоем LSTM для снижения размерности
    * 'linearattention' - модуль многоголового внимания со снижением размерности вектора с помощью линейного слоя (используется для всех экстракторов)
    * 'biaffine' - использование биаффинного метода кодирования слов в последовательности, кроме семантических представлений от BERT.
    * 'selfbiaffine' - использование биаффинного метода, а также механизма внимания. После конкатенации используется линейный слой для уменьшения размерности.

* num_heads - используется для экстракторов, включающих механизм внимания
* mode ('classification') - основная задача классификации
* extractor_use_gcn (Bool) - использование модуля для получения синтаксической информации с помощью парсера от natasha и графового модуля aggcn

In [14]:
model = NNERModel(num_labels=len(ids2labels),
                  device=device,
                  max_seq_len=284,
                  max_span_len=20,
                  num_heads=None,
                  extractor_type='biaffine',
                  mode='classification',
                  extractor_use_gcn=False)
_ = model.to(device)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
# проигнорировать при валидации
optimizer = optim.Adam(model.parameters(),
                        lr=LEARNING_RATE)

total_steps = len(training_loader) * EPOCHS
warmup_steps = int(total_steps * 0.1)
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = warmup_steps,
                                            num_training_steps = total_steps)

In [16]:
# название для сохрания или загрузки модели
safe_prefix = 'testing'

In [17]:
# True при обучении модели с самого начала
# False - загрузка для дообучения
from_start = True

Требуется уточнить путь до места сохранения

По умолчанию сохраняется в папку new_data

Здесь загружается и сохраняется модель и оптимизатор с планировщиков на случай вылета обучения

In [18]:
if from_start == False:
    checkpoint = torch.load(f'new_data/{safe_prefix}_checkpoint.pth.tar')        
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler'])
    resume_epoch = checkpoint['epoch']
    last_best = checkpoint['f_score']
else:
    last_best = 0
    resume_epoch = 0

In [19]:
nner_trainer = GeneralTrainer(model=model,
                              device = device,
                              optimizer=optimizer,
                              scheduler=scheduler,
                              ids2labels=ids2labels,
                              labels2ids=labels2ids)

Весь процесс обучения

По умолчанию лучшая модель сохраняется в папку new_data/

In [ ]:
for epoch in range(EPOCHS):
    if resume_epoch+epoch+1 > EPOCHS:
        break
    print(f'EPOCH {resume_epoch+epoch+1}/{EPOCHS}')
    _ = nner_trainer(training_loader, train_spans, train_inds, mode='train')

    f_score = nner_trainer(validation_loader, val_spans, val_inds, mode='dev')
    
    # на случай отслеживания оценок на валидации и тесте
    #_ = nner_trainer(testing_loader, test_spans, test_inds, mode='test')

    
    if f_score > last_best:
        last_best = f_score
        check_path = f'new_data/{safe_prefix}_checkpoint.pth.tar'
        torch.save({'epoch': epoch+1+resume_epoch,
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'scheduler': scheduler.state_dict(),
                        'f_score' : last_best}, check_path)

##### Валидация на тесте с подгружаемой моделью

In [20]:
test_params = {'batch_size': 1,
                'shuffle': False,
                'num_workers': 0
                }
testing_loader = DataLoader(test_texts_ds, **test_params)

Про выбор гиперпараметров загрузки модели см. ранее

In [21]:
model = NNERModel(num_labels=len(ids2labels),
                  device=device,
                  max_seq_len=284,
                  max_span_len=20,
                  num_heads=None,
                  extractor_type='biaffine',
                  mode='classification',
                  extractor_use_gcn=False)
_ = model.to(device)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [22]:
# путь до сохраненной модели!

# для лучшей модели сохранялось на диск только ее состояние, без оптимизатора и тп
check_path = 'all_data/general_checkpoints/biaffine_checkpoint.pth'
checkpoint = torch.load(check_path)
model.load_state_dict(checkpoint)
# если полное сохранение:
# model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [23]:
nner_trainer = GeneralTrainer(model=model,
                              device=device,
                              ids2labels=ids2labels,
                              labels2ids=labels2ids)

In [26]:
f_score = nner_trainer(testing_loader, test_spans, test_inds, mode='test')

935it [00:54, 17.06it/s]


RESULTS FOR MODE TEST
****************SUB NER TYPES********************
****************GENERAL********************
NATIONALITY F1: 73.10345%
NATIONALITY Recall: 80.30303%
NATIONALITY Precision: 67.08861%
CITY F1: 88.61789%
CITY Recall: 90.83333%
CITY Precision: 86.50794%
TIME F1: 64.00000%
TIME Recall: 66.66667%
TIME Precision: 61.53846%
DATE F1: 91.23810%
DATE Recall: 91.41221%
DATE Precision: 91.06464%
ORGANIZATION F1: 82.00837%
ORGANIZATION Recall: 84.12017%
ORGANIZATION Precision: 80.00000%
COUNTRY F1: 95.31080%
COUNTRY Recall: 96.04396%
COUNTRY Precision: 94.58874%
EVENT F1: 66.19014%
EVENT Recall: 63.16508%
EVENT Precision: 69.51952%
AGE F1: 87.82288%
AGE Recall: 86.23188%
AGE Precision: 89.47368%
NUMBER F1: 85.32110%
NUMBER Recall: 83.78378%
NUMBER Precision: 86.91589%
PRODUCT F1: 75.24752%
PRODUCT Recall: 79.16667%
PRODUCT Precision: 71.69811%
PROFESSION F1: 83.82954%
PROFESSION Recall: 83.68298%
PROFESSION Precision: 83.97661%
FACILITY F1: 63.23529%
FACILITY Recall: 63.23529%